In [1]:
#!/usr/bin/python

# This is the evaluation script for the TREC 2017 RTS evaluation
# (scenario B) with batch NIST assessor judgments, v1.0.

__author__ = 'Luchen'
import argparse
import json
import numpy
import math
import gzip
import os

qrels_path = 'Qrel/rts2017-batch-qrels.txt'
clusters_path = 'Qrel/rts2017-batch-clusters.json'
file_tweet2day = 'Qrel/rts2017-batch-tweets2dayepoch.txt'
ipath='F:/TREC2017/Analysis'
os.chdir(ipath)
path_rn='F:/TREC2017/Analysis/Result/dirichlet'
path_rn='F:/TREC2017/Analysis/Result/bm25'
path_rn='F:/TREC2017/Analysis/Result/jm112'
path_rn='F:/TREC2017/Analysis/Result/bm25-1912'


K = 10
days = []
for i in range(29, 32):
    days.append("201707%02d" % i)
for i in range(1, 6):
    days.append("201708%02d" % i)

    

In [2]:
os.getcwd()

'F:\\TREC2017\\Analysis'

In [3]:
# qrels dictionary, {topic: {tweetid: gain}}
qrels_dt = {}
clusters_day_dt = {}
for line in open(qrels_path).readlines():
    line = line.strip().split()
    topic = line[0]
    tweetid = line[2]
    score = int(line[3])
    if score == -2:
        score = 0
    else:
        score /= float(2)
    if topic in qrels_dt:
        qrels_dt[topic][tweetid] = score
    else:
        qrels_dt[topic] = {tweetid: score}
        clusters_day_dt[topic] = {day: [] for day in days}


# created timestamp and date for each tweetid in the qrel
# tweet2day_dt: {tweetid: date}
tweet2day_dt = {}
for line in open(file_tweet2day).readlines():
    line = line.strip().split()
    tweet2day_dt[line[0]] = line[1]

# cluster dictionaries
clusters_clusterid_dt = {}
clusters_topic_dt = json.load(open(clusters_path))
for topic in clusters_topic_dt:
    clusters_json = clusters_topic_dt[topic]["clusters"]
    if topic not in clusters_clusterid_dt:
        clusters_clusterid_dt[topic] = {}
    for clusterid in clusters_json.keys():
        for tweetid in clusters_json[clusterid]["tweets"]:
            if tweet2day_dt[tweetid] in days:
                clusters_clusterid_dt[topic][tweetid] = clusterid
                clusters_day_dt[topic][tweet2day_dt[tweetid]].append(tweetid)

In [8]:
import glob, os
count =0
os.chdir(path_rn)

fout=open("grid/rdata_new.txt","a+")
fout1=open("grid/result.txt","a+")
minth=0
maxth=0

for file in glob.glob("*.txt"):
    run_path=file
    


    # run dictionaries
    # run_dt: {topic: {date: [tweetids}}
    runname = ''
    run_dt = {}
    run_epoch_dt = {}
    for line in open(run_path, "r").readlines():

        line = line.strip().split()

        runname = line[6]
        topic = line[1]
        if topic in qrels_dt:
            if topic not in run_dt:
                run_dt[topic] = {}
            day = line[0]
            if day not in run_dt[topic]:
                run_dt[topic][day] = []
            tweetid = line[3]
            run_dt[topic][day].append(tweetid)

    print("{0}\t{1:5s}\t{2:6s}\t{3:6s}".format("runtag".ljust(len(runname)), "topic", "nDCGp", "nDCG1"))

    total_ndcg1, total_ndcgp = 0.0, 0.0
    for topic in sorted(qrels_dt):
        topic_ndcg1, topic_ndcgp = 0.0, 0.0
        exist_clusterids = set()
        for day in days:
            interesting = False
            max_gain_dt = {}
            tweets_fromprotocol = clusters_day_dt[topic][day]
            for tweetid in tweets_fromprotocol:
                clusterid = clusters_clusterid_dt[topic][tweetid]
                if clusterid not in exist_clusterids:
                    interesting = True
                    if clusterid not in max_gain_dt:
                        max_gain_dt[clusterid] = qrels_dt[topic][tweetid]
                    else:
                        max_gain_dt[clusterid] = max(max_gain_dt[clusterid], qrels_dt[topic][tweetid])
            if interesting:
                if topic in run_dt and day in run_dt[topic]:
                    ndcg = 0.0
                    gains = []
                    for tweetid in run_dt[topic][day]:
                        gain = 0.0
                        if tweetid in clusters_day_dt[topic][day]:
                            clusterid = clusters_clusterid_dt[topic][tweetid]
                            if clusterid not in exist_clusterids:
                                exist_clusterids.add(clusterid)
                                gain = qrels_dt[topic][tweetid]
                                # tweet gain is with respect to its own gain, not max gain in the cluster
                                # if clusterid in max_gain_dt:
                                #    gain = max_gain_dt[clusterid]
                        gains.append(gain)
                    rank_cut = min(len(gains), K)
                    dcg = 0.0
                    for i in range(rank_cut):
                        gain = gains[i]
                        dcg += float(pow(2, gain) - 1) / math.log(i + 2, 2)

                    # compute idcg
                    top_gains = list(max_gain_dt.values())
                    top_gains.sort(reverse=True)
                    rank_cut = min(len(top_gains), K)
                    idcg = 0.0
                    top_gains = top_gains[:rank_cut]
                    for i in range(rank_cut):
                        gain = top_gains[i]
                        idcg += float(pow(2, gain) - 1) / math.log(i + 2, 2)
                    if idcg != 0:
                        ndcg = dcg / idcg
                    topic_ndcg1 += ndcg
                    topic_ndcgp += ndcg
            else:
                if topic not in run_dt or day not in run_dt[topic]:
                    topic_ndcg1 += 1
                    topic_ndcgp += 1
                elif topic in run_dt and day in run_dt[topic]:
                    push_num = min(K, len(run_dt[topic][day]))
                    topic_ndcgp += (1 - push_num/float(K))



        topic_ndcg1 /= len(days)
        topic_ndcgp /= len(days)
        #print("{0}\t{1:5s}\t{2:.4f}\t{3:.4f}".format(runname, topic, topic_ndcgp, topic_ndcg1))
        #print len(run_path)
        '''
        #diri
        if (len(run_path) == 25) :
            #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
            #jm
            #maxth=int(run_path[13:15])
            #minth=int(run_path[18:20])
            #diri
            maxth=float(run_path[13:17])
            minth=float(run_path[18:21])      
        if (len(run_path) == 24) :
            #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
            #jm
            #maxth=int(run_path[13:15])
            #minth=int(run_path[18:20])
            #diri
            maxth=float(run_path[13:16])
            minth=float(run_path[17:20])
           '''
        if (len(run_path) == 22) :
            #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
            #jm
            #maxth=int(run_path[13:15])
            #minth=int(run_path[18:20])
            #diri
            maxth=int(run_path[9:11])
            minth=int(run_path[14:16])      
        if (len(run_path) == 21) :
            #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
            #jm
            #maxth=int(run_path[13:15])
            #minth=int(run_path[18:20])
            #diri
            maxth=int(run_path[9:11])
            minth=int(run_path[14:15])
        '''
        if (len(run_path) == 26) :
            #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
            #jm
            #maxth=int(run_path[13:15])
            #minth=int(run_path[18:20])
            #diri
            maxth=int(run_path[13:15])
            minth=int(run_path[18:20]) 
        '''
        
        wrstr = topic + " "+ str(round(topic_ndcgp,4)) + " " + str(round(topic_ndcg1,4)) + ' ' +  str(minth)+ ' ' +str(maxth) + '\n'
        fout.write(wrstr)
            #print wrstr
        total_ndcg1 += topic_ndcg1
        total_ndcgp += topic_ndcgp
    total_ndcg1 /= len(qrels_dt)
    total_ndcgp /= len(qrels_dt)
    print("{0}\t{1:5s}\t{2:.4f}\t{3:.4f}".format(runname, "All", total_ndcgp, total_ndcg1))
    tempstr = run_path + " " + str(round(total_ndcgp,4)) + " " + str(round(total_ndcg1,4)) +"\n"
    fout1.write(tempstr)
fout1.close()
fout.close()


runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1909	0.1905
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1973	0.1904
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.2124	0.1864
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1908	0.1908
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1908	0.1908
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1914	0.1914
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1908	0.1908
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1907	0.1903
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1967	0.1904
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.2116	0.1867
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1906	0.1906
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1906	0.1906
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1912	0.1912
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1906	0.1906
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1886	0.1882
runtag    	topic	nDCGp 	nDCG1 
IRLAB-LDRP	All  	0.1943	0.1889
runtag  

In [21]:
len(qrels_dt)
topic=[]
for q in qrels_dt:
    topic.append(q)
len(topic)

97

In [25]:
os.chdir(ipath)
ip=[]
with open('tid95-2.txt','r') as fin:
    line= fin.readlines()
    for l in line:
        l=l.strip().split(' ')
        #tpn=l[0]
        tpid=l[1]
        ip.append(tpid)      
print len(ip)
s=list(set(topic)-set(ip))
print s

95
['RTS192', 'RTS107']


In [7]:
os.chdir(path_rn)

import glob
import os
os.chdir(path_rn)
for file in glob.glob("*.txt"):
    run_path=file
    print (run_path,len(run_path),run_path[9:11],run_path[14:16])

('bm25_0.1 12.0_10.0.txt', 22, '12', '10')
('bm25_0.1 12.0_11.0.txt', 22, '12', '11')
('bm25_0.1 12.0_12.0.txt', 22, '12', '12')
('bm25_0.1 12.0_6.0.txt', 21, '12', '6.')
('bm25_0.1 12.0_7.0.txt', 21, '12', '7.')
('bm25_0.1 12.0_8.0.txt', 21, '12', '8.')
('bm25_0.1 12.0_9.0.txt', 21, '12', '9.')
('bm25_0.1 13.0_10.0.txt', 22, '13', '10')
('bm25_0.1 13.0_11.0.txt', 22, '13', '11')
('bm25_0.1 13.0_12.0.txt', 22, '13', '12')
('bm25_0.1 13.0_6.0.txt', 21, '13', '6.')
('bm25_0.1 13.0_7.0.txt', 21, '13', '7.')
('bm25_0.1 13.0_8.0.txt', 21, '13', '8.')
('bm25_0.1 13.0_9.0.txt', 21, '13', '9.')
('bm25_0.1 14.0_10.0.txt', 22, '14', '10')
('bm25_0.1 14.0_11.0.txt', 22, '14', '11')
('bm25_0.1 14.0_12.0.txt', 22, '14', '12')
('bm25_0.1 14.0_6.0.txt', 21, '14', '6.')
('bm25_0.1 14.0_7.0.txt', 21, '14', '7.')
('bm25_0.1 14.0_8.0.txt', 21, '14', '8.')
('bm25_0.1 14.0_9.0.txt', 21, '14', '9.')
('bm25_0.1 15.0_10.0.txt', 22, '15', '10')
('bm25_0.1 15.0_11.0.txt', 22, '15', '11')
('bm25_0.1 15.0_12.0.tx